# Understanding Your Customers: A Fresh Take on Analyzing Product Reviews

Aisha Al-Khaldi & Huda Joad

## Where are we in the data science pipeline?

- **Ask an interesting question**
- Get the data
- Explore the data
- Model the data
- Communicate/visualize the results

## Our Question

When customers want to express their thoughts and sentiments about a product, they usually do it through written reviews and assign a rating for their experience (typically a numerical rating from 1-5). We would like to delve into the discussion of **the relationship between the content of the review, whether it's positive, negative or neutral and their assigned ratings**.

We will be using xxx API, and try to perform a sentiment analysis on reviews to answer our question.

## Where are we in the data science pipeline?

- Ask an interesting question
- **Get the data**
- Explore the data
- Model the data
- Communicate/visualize the results

In [3]:
# data collection

In [4]:
!pip install -e git+https://github.com/gauravmm/jupyter-testing.git#egg=jupyter-testing

from testing.testing import test
import requests

Obtaining jupyter-testing from git+https://github.com/gauravmm/jupyter-testing.git#egg=jupyter-testing
  Updating c:\users\hudaj\desktop\huda\fall 2023\data science\final-project\src\jupyter-testing clone
  Attempting uninstall: jupyter-testing
    Found existing installation: jupyter-testing 0.0.2
    Uninstalling jupyter-testing-0.0.2:
      Successfully uninstalled jupyter-testing-0.0.2
  Running setup.py develop for jupyter-testing


  Running command git fetch -q --tags
  Running command git reset --hard -q 8c6b703e663a16f77af53a05096f0258274656b2


In [12]:
def retrieve_html_test(retrieve_html):
    status_code, text = retrieve_html("http://www.example.com")
    test.equal(status_code, 200)
    test.true("http://censor.qa/?accessurl=d3d3LmV4YW1wbGUuY29t&urlclassname=&ipaddr=78.100.166.255" in text)

@test
def retrieve_html(url):
    """
    Return the raw HTML at the specified URL.

    Args:
        url (string): 

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """

    r = requests.get(url)
    return r.status_code, r.text

### TESTING retrieve_html: PASSED 2/2
###



## Where are we in the data science pipeline?

- Ask an interesting question
- Get the data
- **Explore the data**
- Model the data
- Communicate/visualize the results

In [6]:
# data processing

In [7]:
# exploration/visualization

## Where are we in the data science pipeline?

- Ask an interesting question
- Get the data
- Explore the data
- **Model the data**
- Communicate/visualize the results

The model we will be using is a Random Forest Classifier. A random forest classifier can capture more complex relationships between words and sentiment. It's a good choice when you want to explore a slightly more advanced model without diving too deep into complex algorithms.

In [8]:
# analysis/machine learning

## Where are we in the data science pipeline?

- Ask an interesting question
- Get the data
- Explore the data
- Model the data
- **Communicate/visualize the results**

In [9]:
# insights

# Delete Later

Questions to answer
- Does the project clearly identify the problem? 
- Does the project clearly describe the relevant data or/and its collection? 
- Does the project clearly explain how the data can be used to draw conclusions about the 
- underlying system? 
- Does the report clearly explain the work that was done? 
- Is the project innovative or novel? 
- Is the model built accurate enough? 
- Does the project use techniques presented in the course (or clearly related to topics 
covered? 
- in the course) to understand and analyze the data for this problem? 
- Does the report explain how this work fits around related work in this subject area? 
- Does the report provide directions for further investigation?